In [1]:
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from datetime import datetime, timedelta
from global_land_mask import globe
import os
import random
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from scipy.interpolate import splprep, splev
import multiprocessing as mp
from tensorflow.keras import backend as K
from IPython.display import clear_output
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import tensorflow as tf


# Constants
RECTANGLE_COORDS = [46.248, 19.859, 64.881, 32.955]
SOLAR_PANEL_AREA = 3000
SOLAR_PANEL_EFFICIENCY = 0.1726
RHO_AIR = 1.225
KITE_AREA = 500
LIFT_COEFFICIENT = 1.0
DRAG_COEFFICIENT = 0.1
SHIP_SPEED = 10.29  # m/s
GRID_SIZE_DEGREES = 0.01
MAX_STEPS_PER_EPISODE = 3945
TIME_INCREMENT_MINUTES = 2.22*60
goal_reward = 1000000
wall_penalty = -20000
energy_consumption_penalty = -100

# Directories for wind and solar data
WIND_DATA_DIR = "mera"
SOLAR_DATA_DIR = "solar energy"

# Load and preprocess MERRA-2 data


def load_and_preprocess_data(date, directory, variable):
    filename = f"MERRA2_400.tavg1_2d_flx_Nx.{date.strftime('%Y%m%d')}.nc4.nc4" if variable in ['ULML', 'VLML'] else f"MERRA2_400.tavg1_2d_rad_Nx.{date.strftime('%Y%m%d')}.nc4_2.nc4"
    filepath = os.path.join(directory, filename)
    dataset = Dataset(filepath)
    data = dataset.variables[variable][:]
    return data

def load_data_parallel(dates, directory, variables):
    pool = mp.Pool(mp.cpu_count())
    results = pool.starmap(load_and_preprocess_data, [(date, directory, variable) for date in dates for variable in variables])
    pool.close()
    pool.join()
    return results

# Calculate apparent wind speed and direction
def calculate_apparent_wind(uwind, vwind, ship_speed, ship_direction):
    wind_speed = np.sqrt(uwind**2 + vwind**2)
    wind_direction = np.arctan2(vwind, uwind)
    apparent_wind_speed = np.sqrt((wind_speed * np.cos(wind_direction - ship_direction) - ship_speed)**2 + (wind_speed * np.sin(wind_direction - ship_direction))**2)
    apparent_wind_direction = np.arctan2(wind_speed * np.sin(wind_direction - ship_direction), wind_speed * np.cos(wind_direction - ship_direction) - ship_speed)
    return apparent_wind_speed, apparent_wind_direction

# Calculate kite power
def calculate_kite_power(apparent_wind_speed, kite_area, lift_coefficient, drag_coefficient):
    lift_force = 0.5 * RHO_AIR * apparent_wind_speed**2 * kite_area * lift_coefficient
    drag_force = 0.5 * RHO_AIR * apparent_wind_speed**2 * kite_area * drag_coefficient
    power_output = (lift_force - drag_force) * apparent_wind_speed
    return power_output

# Calculate solar power output
def calculate_solar_power_output(solar_panel_area, solar_panel_efficiency, swgnt):
    power_output = solar_panel_area * solar_panel_efficiency * swgnt
    return power_output

# Calculate the Haversine distance
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

# Maze environment
class Maze:
    def __init__(self, maze, start_position, goal_position, height, width, lat, lon):
        self.maze = maze
        self.maze_height = height
        self.maze_width = width
        self.start_position = start_position
        self.goal_position = goal_position
        self.lat = lat
        self.lon = lon

    def show_maze(self):
        plt.figure(figsize=(10, 10))
        plt.imshow(self.maze, cmap='gray')
        plt.text(self.start_position[0], self.start_position[1], 'S', ha='center', va='center', color='red', fontsize=20)
        plt.text(self.goal_position[0], self.goal_position[1], 'G', ha='center', va='center', color='green', fontsize=20)
        plt.xticks([]), plt.yticks([])
        plt.show()

# DQN Agent

class DoubleDQNAgent:
    def __init__(self, state_size, action_size, num_episodes):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=20000)
        self.gamma = 0.99  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.num_episodes = num_episodes
        self.learning_rate = 0.001
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(128, input_dim=self.state_size, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state, verbose=0)
            if done:
                target[0][action] = reward
            else:
                t = self.target_model.predict(next_state, verbose=0)
                target[0][action] = reward + self.gamma * np.amax(t[0])
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def adjust_epsilon(self, episode):
        self.epsilon = max(self.epsilon_min, self.epsilon_decay ** episode)

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

# Simulation and Training
def get_state(maze, current_position, goal_position):
    distance_to_goal = haversine_distance(
        maze.lat[current_position[1]], maze.lon[current_position[0]],
        maze.lat[goal_position[1]], maze.lon[goal_position[0]]
    )
    state = np.array([current_position[0], current_position[1], distance_to_goal])
    return state

def finish_episode(agent, maze, current_episode, start_time, wind_data, solar_data, train=True, optimization_goal='distance'):
    current_state = maze.start_position
    is_done = False
    episode_reward = 0
    episode_step = 0
    path = [current_state]
    solar_fluxes = []
    kite_powers = []

    time_increment = timedelta(minutes=TIME_INCREMENT_MINUTES)
    current_time = start_time

    while not is_done and episode_step < MAX_STEPS_PER_EPISODE:
        state = np.reshape(get_state(maze, current_state, maze.goal_position), [1, 3])
        action = agent.act(state)
        next_state = (current_state[0] + actions[action][0], current_state[1] + actions[action][1])

        if (0 <= next_state[0] < maze.maze_width) and (0 <= next_state[1] < maze.maze_height):
            if maze.maze[next_state[1], next_state[0]] == 1:
                reward = wall_penalty
                next_state = current_state
            elif next_state == maze.goal_position:
                path.append(next_state)
                reward = goal_reward - episode_step  # Reward for reaching the goal earlier
                is_done = True
            else:
                path.append(next_state)
                lat1, lon1 = maze.lat[current_state[1]], maze.lon[current_state[0]]
                lat2, lon2 = maze.lat[next_state[1]], maze.lon[next_state[0]]
                lat_end, lon_end = maze.lat[maze.goal_position[1]], maze.lon[maze.goal_position[0]]
                distance_reward = -haversine_distance(lat1, lon1, lat2, lon2) - haversine_distance(lat2, lon2, lat_end, lon_end)

                # Calculate wind and solar effects
                lat_idx, lon_idx = int((lat1 - RECTANGLE_COORDS[1]) / 0.5), int((lon1 - RECTANGLE_COORDS[0]) / 0.625)
                uwind = wind_data['uwind'][:, lat_idx, lon_idx]
                vwind = wind_data['vwind'][:, lat_idx, lon_idx]
                swgnt = solar_data['swgnt'][:, lat_idx, lon_idx]

                apparent_wind_speed, apparent_wind_direction = calculate_apparent_wind(uwind, vwind, SHIP_SPEED, np.arctan2(lon2 - lon1, lat2 - lat1))
                kite_power = calculate_kite_power(apparent_wind_speed, KITE_AREA, LIFT_COEFFICIENT, DRAG_COEFFICIENT)
                solar_power_output = calculate_solar_power_output(SOLAR_PANEL_AREA, SOLAR_PANEL_EFFICIENCY, swgnt)

                # Calculate energy consumption
                energy_gain = kite_power.mean() + solar_power_output.mean()
                energy_reward = energy_consumption_penalty * energy_gain

                if optimization_goal == 'distance':
                    reward = distance_reward
                elif optimization_goal == 'energy':
                    reward = energy_reward
                else:
                    reward = distance_reward + energy_reward

                # Add a small step penalty to encourage faster goal reaching
                step_penalty = -1
                reward += step_penalty

                # Record values for plotting later
                solar_fluxes.append(solar_power_output.mean())
                kite_powers.append(kite_power.mean())

                # Update time
                current_time += time_increment

        else:
            reward = wall_penalty
            next_state = current_state

        episode_reward += reward
        episode_step += 1

        if train:
            next_state_reshaped = np.reshape(get_state(maze, next_state, maze.goal_position), [1, 3])
            agent.remember(state, action, reward, next_state_reshaped, is_done)

        current_state = next_state

    if not is_done:
        print(f"Episode {current_episode + 1} reached max steps without reaching the goal.")

    return episode_reward, episode_step, path, solar_fluxes, kite_powers

def calculate_total_energy_consumption(path, start_time, wind_data, solar_data):
    current_time = start_time
    total_energy = 0

    for i in range(1, len(path)):
        lat1, lon1 = maze.lat[path[i-1][1]], maze.lon[path[i-1][0]]
        lat2, lon2 = maze.lat[path[i][1]], maze.lon[path[i][0]]
        distance = haversine_distance(lat1, lon1, lat2, lon2)

        # Calculate the time taken to travel this distance
        travel_time = distance / (SHIP_SPEED / 1000)  # converting speed to km/h

        # Calculate wind and solar effects for each hour in the travel time
        for _ in range(int(travel_time)):
            try:
                lat_idx, lon_idx = int((lat1 - RECTANGLE_COORDS[1]) / 0.5), int((lon1 - RECTANGLE_COORDS[0]) / 0.625)
                uwind = wind_data['uwind']
                vwind = wind_data['vwind']
                swgnt = solar_data['swgnt']

                uwind_region = uwind[:, lat_idx, lon_idx]
                vwind_region = vwind[:, lat_idx, lon_idx]
                swgnt_region = swgnt[:, lat_idx, lon_idx]
            
                apparent_wind_speed, apparent_wind_direction = calculate_apparent_wind(uwind_region, vwind_region, SHIP_SPEED, np.arctan2(lon2 - lon1, lat2 - lat1))
                kite_power = calculate_kite_power(apparent_wind_speed, KITE_AREA, LIFT_COEFFICIENT, DRAG_COEFFICIENT)
                solar_power_output = calculate_solar_power_output(SOLAR_PANEL_AREA, SOLAR_PANEL_EFFICIENCY, swgnt_region)

                total_energy += kite_power.mean() + solar_power_output.mean()

            except FileNotFoundError:
                break  # Stop if data is not available for the next hour

            current_time += timedelta(hours=1)
    
    print(f"Total energy consumption for the path: {total_energy} J")
    return total_energy

def test_agent(agent, maze, start_time, wind_data, solar_data, num_episodes=1):
    paths = []
    all_solar_fluxes = []
    all_kite_powers = []
    for episode in range(num_episodes):
        episode_reward, episode_step, path, solar_fluxes, kite_powers = finish_episode(agent, maze, episode, start_time, wind_data, solar_data, train=False)
        print(f"Episode {episode + 1}: Learned Path:")
        for row, col in path:
            print(f"({row}, {col})-> ", end='')
        print("Goal!")

        print(f"Episode {episode + 1}: Number of steps:", episode_step)
        print(f"Episode {episode + 1}: Total reward:", episode_reward)

        paths.append(path)
        all_solar_fluxes.append(solar_fluxes)
        all_kite_powers.append(kite_powers)

    paths = np.array(paths)
    np.save("path.npy", paths)

    return paths, all_solar_fluxes, all_kite_powers

def train_agent(agent, maze, start_time, wind_data, solar_data, num_episodes=200, optimization_goal='distance'):
    episode_rewards = []
    episode_steps = []
    best_path = []
    best_solar_fluxes = []
    best_kite_powers = []
    best_reward = -float('inf')
    batch_size = 32

    for episode in range(num_episodes):
        agent.adjust_epsilon(episode)
        episode_reward, episode_step, path, solar_fluxes, kite_powers = finish_episode(agent, maze, episode, start_time, wind_data, solar_data, train=True, optimization_goal=optimization_goal)
        episode_rewards.append(episode_reward)
        episode_steps.append(episode_step)

        if episode_reward > best_reward:
            best_reward = episode_reward
            best_path = path
            best_solar_fluxes = solar_fluxes
            best_kite_powers = kite_powers

        # Print progress every episode
        print(f"Episode {episode + 1} - Steps: {episode_step}, Reward: {episode_reward}")

        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

        # Update target model every 10 episodes
        if episode % 10 == 0:
            agent.update_target_model()

    print(f"Best Reward: {best_reward}")
    print(f"Best Path: {best_path}")
    print(f"Best Solar Fluxes: {best_solar_fluxes}")
    print(f"Best Kite Powers: {best_kite_powers}")

    plt.figure(figsize=(15, 10))

    plt.subplot(2, 2, 1)
    plt.plot(episode_rewards)
    plt.xlabel('Episode')
    plt.ylabel('Cumulative Reward')
    plt.title('Reward per Episode')
    average_reward = sum(episode_rewards) / len(episode_rewards)
    print(f"The average reward is: {average_reward}")

    plt.subplot(2, 2, 2)
    plt.plot(episode_steps)
    plt.xlabel('Episode')
    plt.ylabel('Steps Taken')
    plt.title('Steps per Episode')
    average_steps = sum(episode_steps) / len(episode_steps)
    print(f"The average steps is: {average_steps}")

    plt.subplot(2, 2, 3)
    plt.imshow(maze.maze, cmap='gray')
    plt.text(maze.start_position[0], maze.start_position[1], 'S', ha='center', va='center', color='red', fontsize=20)
    plt.text(maze.goal_position[0], maze.goal_position[1], 'G', ha='center', va='center', color='green', fontsize=20)
    for row, col in best_path:
        plt.text(row, col, "x", va='center', color='blue', fontsize=5)
    plt.xticks([]), plt.yticks([])
    plt.grid(color='black', linewidth=2)
    plt.title('Final Path Learned')

    plt.subplot(2, 2, 4)
    plt.plot(best_solar_fluxes, label='Solar Flux (W/m^2)')
    plt.plot(best_kite_powers, label='Kite Power (W)')
    plt.xlabel('Step')
    plt.ylabel('Value')
    plt.title('Solar Flux and Kite Power per Step')
    plt.legend()

    plt.tight_layout()
    plt.show()

    if best_path:
        high_res_path = [(int(p[0] * 0.5 / GRID_SIZE_DEGREES), int(p[1] * 0.625 / GRID_SIZE_DEGREES)) for p in best_path]

        high_res_path = [(min(len(maze.lon) - 1, max(0, p[0])), min(len(maze.lat) - 1, max(0, p[1]))) for p in high_res_path]

        x_coords = [maze.lon[p[0]] for p in high_res_path]
        y_coords = [maze.lat[p[1]] for p in high_res_path]

        if len(x_coords) < 2 or len(y_coords) < 2:
            print("Not enough points for spline fitting.")
            return

        tck, u = splprep([x_coords, y_coords], s=0)
        x_smooth, y_smooth = splev(np.linspace(0, 1, 1000), tck)

        origin = [49.469, 29.362]
        destination = [58.836, 24.385]
        if origin[0] > destination[0]:
            origin, destination = destination, origin

        lat = np.arange(19.859, 32.955, 0.01)
        lon = np.arange(46.248, 64.881 + 0.625, 0.01)
        lon_grid, lat_grid = np.meshgrid(lon, lat)
        z = globe.is_land(lat_grid, lon_grid)
        z = np.array(z[::-1].astype(int))

        plt.figure(figsize=(10, 10))
        plt.imshow(z, cmap='gray')
        plt.plot(y_smooth, x_smooth, 'r-', label='Smoothed Path')
        plt.scatter([maze.lon[maze.start_position[0]]], [maze.lat[maze.start_position[1]]], c='red', label='Start')
        plt.scatter([maze.lon[maze.goal_position[0]]], [maze.lat[maze.goal_position[1]]], c='green', label='Goal')
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')
        plt.title('Final Smoothed Path')
        plt.legend()
        plt.show()

# Initialize environment and agent
origin = [49.469, 29.362]
destination = [58.836, 24.385]
if origin[0] > destination[0]:
    origin, destination = destination, origin

lat = np.arange(19.859, 32.955, 0.5)
lon = np.arange(46.248, 64.881+0.625,0.625 )
lon_grid, lat_grid = np.meshgrid(lon, lat)
z = globe.is_land(lat_grid, lon_grid)
z = np.array(z[::-1].astype(int))

test_lat = lat - origin[1]
test_lat = test_lat[::-1]
test_lon = lon - origin[0]
start_pos_lat = np.where(abs(test_lat) == min(abs(test_lat)))
start_pos_lon = np.where(abs(test_lon) == min(abs(test_lon)))
startpos = (start_pos_lon[0][0], start_pos_lat[0][0])

test_lat = lat - destination[1]
test_lat = test_lat[::-1]
test_lon = lon - destination[0]
destination_pos_lat = np.where(abs(test_lat) == min(abs(test_lat)))
destination_pos_lon = np.where(abs(test_lon) == min(abs(test_lon)))
destinationpos = (destination_pos_lon[0][0], destination_pos_lat[0][0])
lat = lat[::-1]
lon = lon[::-1]

maze = Maze(z, startpos, destinationpos, len(z), len(z[0]), lat, lon)
maze.show_maze()
actions = [(-1, 0), (1, 0), (0, -1), (0, 1), (1, 1), (1, -1), (-1, 1), (-1, -1)]




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\parsh\PycharmProjects\pythonProject6\.conda\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\parsh\PycharmProjects\pythonProject6\.conda\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\parsh\PycharmProjects\pythonProject6\.conda\Lib\site-packages\ipykernel\kernelapp.py", line 7

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\parsh\PycharmProjects\pythonProject6\.conda\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\parsh\PycharmProjects\pythonProject6\.conda\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\parsh\PycharmProjects\pythonProject6\.conda\Lib\site-packages\ipykernel\kernelapp.py", line 7

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\parsh\PycharmProjects\pythonProject6\.conda\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\parsh\PycharmProjects\pythonProject6\.conda\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\parsh\PycharmProjects\pythonProject6\.conda\Lib\site-packages\ipykernel\kernelapp.py", line 7

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

In [ ]:

state_size = 3
action_size = len(actions)
num_episodes = 50
agent = DoubleDQNAgent(state_size, action_size, num_episodes)


c:\Users\parsh\PycharmProjects\pythonProject6\.conda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [2]:
start_time = datetime(2023, 1, 1)

# Preload wind and solar data
dates = [start_time + timedelta(days=i) for i in range(num_episodes)]
wind_data = {
    'uwind': load_and_preprocess_data(start_time, WIND_DATA_DIR, 'ULML'),
    'vwind': load_and_preprocess_data(start_time, WIND_DATA_DIR, 'VLML')
}
solar_data = {
    'swgnt': load_and_preprocess_data(start_time, SOLAR_DATA_DIR, 'SWGNT')
}

# Training the agent
train_agent(agent, maze, start_time, wind_data, solar_data, num_episodes=num_episodes, optimization_goal='energy')


NameError: name 'num_episodes' is not defined